In [27]:
import pandas as pd
import requests
import pickle

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [28]:
spotify_client_id = 'e0e51ec7feeb45b89b165ae1007c535d'
spotify_client_secret = 'cf444557e7ad446ebd5c2d4cc6a781cf'

In [29]:
client_credentials_manager = SpotifyClientCredentials(spotify_client_id, spotify_client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [30]:
labels_df = pd.read_excel('MoodyLyrics/ml_raw.xlsx')
labels_df

,Index,Artist,Title,Mood
0,ML1,Usher,There Goes My Baby,relaxed
1,ML2,Da'Ville,On My Mind,relaxed
2,ML3,Rihanna,Rockstar 101,relaxed
3,ML4,J. Holiday,Bed,relaxed
4,ML5,Morgan Heritage,Don't Haffi Dread,angry
...,...,...,...,...
2590,ML2591,Slayer,Fight Till Death,angry
2591,ML2592,Anthrax,Among the Living,angry
2592,ML2593,Opeth,Heir Apparent,sad
2593,ML2594,Sepultura,Territory,angry


In [31]:
def transform(album_info, artist_info, track_info, audio_analysis, audio_features):
    
    album_artists = []
    list_of_artists = album_info['artists']
    for artist in list_of_artists:
        album_artists.append(artist['name'])

    album_tracks = list()
    for track in album_info['tracks']['items']:
        album_tracks.append(track['id'])

    album = pd.DataFrame()
    album.at['album_id', 0] = album_info['id']
    album.at['album_name', 0] = album_info['name']
    album.at['album_type', 0] = album_info['type']
    album.at['number_of_tracks', 0] = album_info['total_tracks']
    album.at['album_artists', 0] = ' & '.join(album_artists)
    album.at['track_ids', 0] = album_tracks
    album.at['album_popularity', 0] = album_info['popularity']
    album.at['album_available_markets', 0] = album_info['available_markets']
    album.at['album_href', 0] = album_info['href']
    album.at['album_genres', 0] = album_info['genres']
    album.at['album_label', 0] = album_info['label']
    album.at['album_images', 0] = album_info['images']
    album.at['album_release_date', 0] = album_info['release_date']
    album = album.transpose()
    
    artist = pd.DataFrame()
    artist.at['artist_id', 0] = artist_info['id']
    artist.at['artist_name', 0] = artist_info['name']
    artist.at['artist_followers', 0] = artist_info['followers']['total']
    artist.at['artist_popularity', 0] = artist_info['popularity']
    artist.at['artist_genres', 0] = artist_info['genres']
    artist.at['artist_href', 0] = artist_info['href']
    artist.at['artist_images', 0] = artist_info['images']
    artist.at['artist_type', 0] = artist_info['type']
    artist = artist.transpose()
    
    featuring_artists = list()
    for a in track_info['artists']:
        if a['name'] != artist_info['name']:
            featuring_artists.append(a['name'])

    track_information = pd.DataFrame()
    if featuring_artists:
        track_information.at['featuring_artists', 0] = str(featuring_artists)
    else:
        track_information.at['featuring_artists', 0] = None

    track_information.at['track_name', 0] = track_info['name']
    track_information.at['track_popularity', 0] = track_info['popularity']
    track_information.at['available_markets', 0] = track_info['available_markets']

    aa = pd.DataFrame.from_dict(audio_analysis['track'], orient = 'index')
    af = pd.DataFrame.from_dict(audio_features[0], orient = 'index')

    track = pd.concat([aa, af, track_information]).transpose()
    
    return {
        'album': album,
        'artist': artist,
        'track': track
    }

In [32]:
def extract():
    albums = pd.DataFrame()
    artists = pd.DataFrame()
    tracks = pd.DataFrame()
    for i, row in labels_df.iterrows():

        try:
            artist_name = row['Artist']
            track_name = row['Title']
            print(i, artist_name, track_name)

            match = sp.search(q='artist:' + artist_name + ' track:' + track_name, type='track')
            album_id = match['tracks']['items'][0]['album']['id']
            artist_id = match['tracks']['items'][0]['artists'][0]['id']
            track_id = match['tracks']['items'][0]['id']

            album_info = sp.album(album_id)
            artist_info = sp.artist(artist_id)
            track_info = sp.track(track_id)

            audio_analysis = sp.audio_analysis(track_id)
            audio_features = sp.audio_features(track_id)

            drop_cols = [
                'duration', 'codestring', 'code_version', 'echoprintstring', 
                 'echoprint_version', 'synchstring', 'synch_version', 
                 'rhythmstring', 'rhythm_version', 'num_samples', 'sample_md5', 
                 'offset_seconds', 'window_seconds', 'analysis_sample_rate', 
                 'analysis_channels', 'end_of_fade_in', 'start_of_fade_out'
                    ]

            for key in drop_cols:
                del audio_analysis['track'][key]

            for key in ['analysis_url', 'uri']:
                del audio_features[0][key]

            out = transform(album_info, artist_info, track_info, audio_analysis, audio_features)

            albums = pd.concat([albums, out['album']]).drop_duplicates(subset = ['album_id'])
            artists = pd.concat([artists, out['artist']]).drop_duplicates(subset = ['artist_id'])
            tracks = pd.concat([tracks, out['track']]).drop_duplicates(subset = ['id'])

        except Exception as e:
            print(e)
            
    return {'albums': albums, 'artists': artists, 'tracks': tracks}

In [33]:
tables = extract()

0 Usher There Goes My Baby
1 Da'Ville On My Mind
list index out of range
2 Rihanna Rockstar 101 
3 J. Holiday Bed
4 Morgan Heritage Don't Haffi Dread
list index out of range
5 Whitney Houston My Love Is Your Love
6 Serani Study People
7 Austra Spellwork
8 Coro Can't Let You Go
list index out of range
9 Beenie Man Slam
10 Deitrick Haddon Love Him Like I Do 
11 The Jackson 5 ABC
12 Maysa The Lady In My Life
13 Ryan Shaw We Got Love
14 The Paris Sisters I Love How You Love Me
15 New Direction I Came To Jesus
16 Kurt Carr & The Kurt Carr Singers God Great God
17 Kenny Lattimore And I Love Her
18 Jordin Sparks Battlefield
19 Spoon You Got Yr. Cherry Bomb
20 Tarralyn Ramsey Peace Of God
21 Tyrese One
22 Dr. Dog Where'd All The Time Go?
list index out of range
23 Tarralyn Ramsey Unconditional Love
24 Kool & The Gang Steppin' Into Love
list index out of range
25 Anders Osborne Darkness At The Bottom
26 The Rolling Stones Gimme Shelter
27 The Rolling Stones Happy
28 Bobby Bare Detroit City
29 G

252 Tank Sex Music
253 Matt Redman This Is How We Know
254 Big Daddy Weave Just The Way I Am
255 Point Of Grace All The World
256 Steven Curtis Chapman All About Love
257 Chris Tomlin Our God
258 Steven Curtis Chapman Not Home Yet
259 Digital Underground Kiss You Back
260 Mark Ronson Bang Bang Bang 
261 Sting I Hung My Head
262 Carrie Rodriguez Big Love
263 The Secret Sisters The One I Love Is Gone
264 Shelby Lynne Family Tree
265 Ra Ra Riot Foolish
266 Nitin Sawhney Sunset
267 Sidewalk Prophets You Can Have Me
268 Rebecca St. James Song Of Love
269 Aaron Shust My Savior My God
270 By The Tree Beautiful One
271 Josh Wilson Before The Morning
272 Aaron Shust Give Me Words To Speak
273 Jeremy Camp Take You Back
274 Jeremy Camp Beautiful One
275 Rich Mullins Awesome God
276 City On A Hill God Of Wonders
list index out of range
277 Phillips, Craig & Dean Friend Of God
278 Third Day I've Always Loved You
list index out of range
279 Building 429 I Believe 
280 DecembeRadio Love Can
281 Avalo

489 The Georgia Mass Choir Joy
490 Stevie Wonder Lookin' For Another Pure Love
491 Jody Watley Real Love
492 Chet Baker Let's Get Lost
493 Nina Simone Feeling Good
494 Mel Torme Comin' Home Baby
495 Freddie Hubbard Red Clay
496 Cassandra Wilson Time After Time
497 Patrice Rushen Remind Me
498 Curtis Mayfield Give Me Your Love 
499 Chayanne Tu Boca
500 Jeremias Hay Un Amor Afuera
501 David Bisbal Esclavo De Sus Besos
502 Darlene Zschech Call Upon His Name 
503 Aaron Shust To God Alone
504 Laura Story What A Savior
505 Darrell Evans Trading My Sorrows
506 Newsong My Heart's Already There
507 Britt Nicole The Lost Get Found
508 Jadon Lavik Hear Our Song
509 The Afters All That I Am
510 Todd Agnew Joy Unspeakable
511 Norman Hutchins Get Ready For Your Miracle
512 Diana Krall You're My Thrill
list index out of range
513 Jonathan Butler Color Green
514 John Coltrane Acknowledgement
515 Seether Driven Under
516 Soundgarden Rhinosaur
517 Metallica Fight Fire With Fire
518 Deftones You've Seen 

731 Intocable Contra Viento Y Marea
732 Sonny James Young Love
733 Dolly Parton Yellow Roses
734 Innerpartysystem American Trash
735 Edward Sharpe & The Magnetic Zeros Home
736 Tenacious D The Metal
737 The Heartbeats A Thousand Miles Away
738 Surface Happy
739 The Gap Band Yearning For Your Love
740 War Outlaw
741 Dinora Y La Juventud A Puro Dolor
742 Lhasa De Cara A La Pared
743 Duncan Dhu En Algun Lugar
744 Miguel Aceves Mejia Rogaciano
745 Audioslave Original Fire
746 Chickenfoot Sexy Little Thing
747 Veruca Salt Seether
748 Corinne Bailey Rae Is This Love
749 Jack White Wayfaring Stranger
750 Cross Canadian Ragweed Fightin' For
751 Janie Frickie It Ain't Easy Bein' Easy
list index out of range
752 The Statler Brothers Bed Of Roses
753 Dolly Parton But You Know I Love You
754 Marilyn Manson The Nobodies
755 Ozzy Osbourne Killer Of Giants
756 L.A. Guns Never Enough
757 Iron Maiden Iron Maiden
758 Honeymoon Suite New Girl Now
759 Stevie Ray Vaughan & Double Trouble Willie The Wimp 
7

966 Jane Monheit I'm Glad There Is You
list index out of range
967 Ann Hampton Callaway What Is This Thing Called Love?
list index out of range
968 B.B. King Don't Answer The Door
list index out of range
969 James Hunter No Smoke Without Fire
970 Melody Gardot Over The Rainbow
971 Stevie Wonder For Your Love
972 Simon & Garfunkel Homeward Bound
973 Wadsworth Mansion Sweet Mary
974 Bayside Sick, Sick, Sick
975 Does It Offend You, Yeah? The Monkeys Are Coming
976 Rosie & The Originals Angel Baby
977 Maze Happy Feelin's 
978 Zoe Love
979 Young Galaxy The Angels Are Surely Weeping
980 PJ Harvey This Mess We're In
981 Kylesa Dust
982 Silversun Pickups Waste It On
983 Oh Land White Nights
984 The Beatles And I Love Her
985 Serge Gainsbourg Requiem Pour Un C...
986 Bebel Gilberto Aganju
987 The Dramatics Get Up And Get Down
988 Tinsley Ellis Speak No Evil
989 Kurt Elling Matte Kudasai
990 Merle Haggard Going Where The Lonely Go
991 Johnny Cash Rusty Cage
992 Green Day Sassafrass Roots
list in

...

1644 Anti-Flag Turncoat
1645 Dottie Peoples Happy In Jesus
1646 CeCe Winans Away In A Manger
1647 Brian McKnight Because Of His Love
1648 Israel & New Breed Have Yourself A Merry Little Christmas
1649 Daryl Hall & John Oates Home For Christmas
1650 Harry Connick, Jr. Blue Christmas
1651 Kenny G Joy To The World
1652 Peter White Silent Night
1653 Vanessa Williams Joy To The World 
1654 Frank Sinatra Have Yourself A Merry Little Christmas
1655 Mariah Carey Joy To The World
1656 Mariah Carey Silent Night
1657 Rascal Flatts I'll Be Home For Christmas
list index out of range
1658 Kenny Rogers & Dolly Parton With Bells On
list index out of range
1659 Alan Jackson Let It Be Christmas
1660 Kenny Chesney Pretty Paper
1661 Jo Dee Messina Silent Night
1662 The Carpenters Have Yourself A Merry Little Christmas
list index out of range
1663 Casting Crowns Silent Night
1664 Jim Brickman Sending You A Little Christmas
1665 Lanae' Hale Carol Of The Bells
1666 James Taylor Have Yourself A Merry Little C

1865 Neil Diamond Kentucky Woman
1866 Tinariwen Matadjem Yinmixan
1867 Charles Bradley Heart Of Gold 
1868 Joan Armatrading Love And Affection
1869 Blood Brothers Love Rhymes With Hideous Car Wreck
1870 Miike Snow Burial
1871 Alice Russell Hurry On Now
1872 Dixie Chicks I Can Love You Better
list index out of range
1873 Darryl Worley Family Tree
1874 Chris Ledoux Hooked On An 8 Second Ride
1875 The Brian Jonestown Massacre Wisdom
1876 Brooks & Dunn We'll Burn That Bridge
list index out of range
1877 The Ting Tings That's Not My Name
1878 John Gold Baby It's Your Life
1879 Richie Havens Here Comes The Sun
1880 Sophie Milman I'm On Fire
list index out of range
1881 Metallica Seek &#x26; Destroy
list index out of range
1882 Radiohead Nude
1883 Crocodiles All My Hate And My Hexes Are For You
1884 Austra Spellwork
1885 The Suburbs Love Is The Law
1886 The Kinks Everybody's Gonna Be Happy
list index out of range
1887 VHS Or Beta Love In My Pocket
1888 The Flaming Lips Sagittarius Silver Anno

2076 Iggy & The Stooges Gimme Danger
2077 Rachelle Ferrell Extensions
list index out of range
2078 Cassandra Wilson Black Orpheus
list index out of range
2079 Gregg Allman My Love is Your Love
2080 JJ Grey & Mofro Lullaby
2081 Bettye Lavette Joy
2082 Cowboy Junkies Flirted With You All My Life
2083 Nitzer Ebb Join In The Chant
2084 Paris Hilton Fightin' Over Me 
2085 Killing Joke The Death And Resurrection Show
2086 Eskmo Starships
2087 Film School Two Kinds
2088 Jet Black Hearts 
2089 Rev Theory Broken Bones
2090 Cold Suffocate
2091 Nick Lowe Without Love
2092 Slayer Evil Has No Boundaries
2093 Emilio Navaira No Es El Fin Del Mundo 
2094 Queen Play The Game
2095 Cafe Tacuba Las Persianas
list index out of range
2096 Weekend End Times
2097 Exene Cervenka Beyond You
2098 The Autumn Defense Step Easy
2099 The Beatles Free As A Bird
2100 Irma Thomas Same Old Blues
2101 Jimi Hendrix Who Knows
2102 Cloud Nothings Old Street
2103 Vivian Girls All The Time
2104 The Avalanches Flight Tonight
2

2291 A Place To Bury Strangers Everything Always Goes Wrong
2292 Gorillaz Bill Murray
2293 Sherwood Free
2294 Skeletonwitch Where The Light Has Failed
2295 Thomas Dolby Dissidents (The Search For Truth Part 1) (2009 Digital Remaster)
list index out of range
2296 Neil Sedaka Because Of You
2297 Keb' Mo' Dangerous Mood
2298 Jimi Hendrix Experience Fire
2299 22-20s Hold On
2300 Wyclef Jean Midnight Lovers
2301 evanescence my_immortal
list index out of range
2302 Minnie Riperton Completeness
2303 Iggy Pop Lust for life (recorded during the us tour in 1986)
2304 The Beatles All You Need Is Love
2305 The Gathering Locked Away
2306 Zeromancer The Death Of Romance
2307 System of a Down War?
2308 Keb' Mo' The Door
2309 Usher I'll Show You Love
list index out of range
2310 Junior Kimbrough Meet Me in the City
2311 Bass  Fontella Rescue Me
2312 Black Flag Police Story
2313 Rancid Black Lung (Album Version)
list index out of range
2314 The Shirelles Will You Love Me Tomorrow
2315 The Beatles In My

2478 throwing_muses hate_my_way
list index out of range
2479 Small Faces Itchycoo Park
2480 Hot Tuna Candy Man
2481 eels cancer_for_the_cure
list index out of range
2482 Mink DeVille Heaven Stood Still
2483 Owens  Buck Sally Was a Good Old Girl
2484 Blind Willie Johnson God Don't Never Change
2485 Dean Martin Until You Love Someone
2486 The Beatles And I Love Her
2487 Hot Tuna Death Don't Have No Mercy
2488 Pop  Iggy Lust for Life
2489 Asleep At The Wheel Let Me Go Home Whiskey
2490 The Beatles Maggie Mae
2491 The Killers A White Demon Love Song (Soundtrack Album Version)
list index out of range
2492 Panic At The Disco Northern Downpour (Album Version)
list index out of range
2493 Bad Religion The Lie
2494 Phantom Planet Here I Am
2495 The Beatles I Should Have Known Better
2496 SNAP! The World In My Hands (We Are One)
2497 The Police So Lonely
2498 Atreyu Ex's And Oh's (Instrumental Version)
list index out of range
2499 Steel Pulse We Can Do It
2500 Kathy Troccoli Never My Love
2501 D

In [34]:
tables['albums']

,album_id,album_name,album_type,number_of_tracks,album_artists,track_ids,album_popularity,album_available_markets,album_href,album_genres,album_label,album_images,album_release_date
0,6A1F3Fkq5dYeYYNkXflcTX,Raymond v Raymond (Expanded Edition),album,22,Usher,"[3IfytpNFSJwGXrSJOVhKLV, 030OCtLMrljNhp8OWHBWW...",75,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",https://api.spotify.com/v1/albums/6A1F3Fkq5dYe...,[],LaFace Records,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",2010-03-30
0,7uGmyYwDFJbSc1xs4hkEs2,Rated R,album,13,Rihanna,"[4oHkmDMdEHbd0f1xWqyFC7, 59QZX1c9i8Sn1hdVEF6IU...",71,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",https://api.spotify.com/v1/albums/7uGmyYwDFJbS...,[],Def Jam Recordings,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",2009-11-20
0,1Ypwht853ZVmsplg2DnZLI,Back Of My Lac',album,14,J. Holiday,"[17ygLBycpMJk3EdZ4p80h5, 5wXetHpKLjwYf3QkQLmiI...",60,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",https://api.spotify.com/v1/albums/1Ypwht853ZVm...,[],Capitol Records,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",2007-01-01
0,00NABajpGsPCObfcl4LJsM,My Love Is Your Love,album,13,Whitney Houston,"[7861kBrZ5aw7o4NfggcyYW, 4J9u7UsXH7F6sFI32KnYu...",69,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",https://api.spotify.com/v1/albums/00NABajpGsPC...,[],Arista,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",1998-11-17
0,2DRsSArHxJzseNGydpFJ5n,The Hit List,album,25,Various Artists,"[4M4d6ZWqorPfUoyegzja90, 6LQrgsTv9OszIIIQx42Ij...",18,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",https://api.spotify.com/v1/albums/2DRsSArHxJzs...,[],Tad's Record,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",2009-04-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,6Dee9XTTIUh5zXRUguuC40,Undercover (2009 Re-Mastered),album,10,The Rolling Stones,"[4N8oxV3hvn8Efyo4yxp3LG, 7hUlZfirjFpQwzoE5GFwy...",41,"[AD, AL, AR, AT, AU, BA, BE, BG, BO, BR, BY, C...",https://api.spotify.com/v1/albums/6Dee9XTTIUh5...,[],Polydor Records,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",1983-11-07
0,4ypcv69cLuV6HX8XUH0c5U,At War With Satan (Bonus Track Edition),album,15,Venom,"[6QaGCSR5UDWAVFA1QyNw0d, 2EHSnBK3K7cwZfMTKDjQm...",29,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",https://api.spotify.com/v1/albums/4ypcv69cLuV6...,[],Noise Records,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",1984-01-01
0,0gOV46yCRVgdehR1SnFtuy,Wingspan,album,40,Paul McCartney,"[3hFYMKAolw99FZMXEGgGwv, 0aCwxQS9Okv3683jgntcw...",50,"[CA, US]",https://api.spotify.com/v1/albums/0gOV46yCRVgd...,[],Paul McCartney Catalog,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",2001-05-07
0,0fayDgcekIaW0yQtUQ8CDm,Watershed (Special Edition),album,10,Opeth,"[3P0LXXybFHgu2aY1NyGViV, 6OGkZoqso1wbIy5O6cV56...",50,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",https://api.spotify.com/v1/albums/0fayDgcekIaW...,[],Roadrunner Records,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",2008-06-03


In [35]:
tables['artists']

,artist_id,artist_name,artist_followers,artist_popularity,artist_genres,artist_href,artist_images,artist_type
0,23zg3TcAtWQy7J6upgbUnj,Usher,8314056,83,"[atl hip hop, dance pop, pop, r&b, south carol...",https://api.spotify.com/v1/artists/23zg3TcAtWQ...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",artist
0,5pKCCKE2ajJHZ9KAiaK11H,Rihanna,40466639,90,"[barbadian pop, dance pop, pop, pop dance, pos...",https://api.spotify.com/v1/artists/5pKCCKE2ajJ...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",artist
0,7G6hXrjGpi6I7waNl4wxAk,J. Holiday,1088116,56,"[bassline, hip pop, new jack swing, pop r&b, p...",https://api.spotify.com/v1/artists/7G6hXrjGpi6...,"[{'height': 1000, 'url': 'https://i.scdn.co/im...",artist
0,6XpaIBNiVzIetEPCWDvAFP,Whitney Houston,6135084,81,"[dance pop, pop, urban contemporary]",https://api.spotify.com/v1/artists/6XpaIBNiVzI...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",artist
0,22099GssG7Ood0d5UIh2wo,Serani,98524,57,"[dancehall, reggae fusion]",https://api.spotify.com/v1/artists/22099GssG7O...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",artist
...,...,...,...,...,...,...,...,...
0,2FjkZT851ez950cyPjeYid,Bobby McFerrin,141627,59,[a cappella],https://api.spotify.com/v1/artists/2FjkZT851ez...,"[{'height': 1000, 'url': 'https://i.scdn.co/im...",artist
0,3dkbV4qihUeMsqN4vBGg93,Al Green,1382357,68,"[classic soul, funk, memphis soul, quiet storm...",https://api.spotify.com/v1/artists/3dkbV4qihUe...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",artist
0,3Mcii5XWf6E0lrY3Uky4cA,Ice Cube,4104721,74,"[conscious hip hop, g funk, gangster rap, hard...",https://api.spotify.com/v1/artists/3Mcii5XWf6E...,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",artist
0,5fwaejlOHVBAw1KhIPPaQe,Venom,307479,47,"[black metal, black thrash, british black meta...",https://api.spotify.com/v1/artists/5fwaejlOHVB...,"[{'height': 1000, 'url': 'https://i.scdn.co/im...",artist


In [36]:
tables['tracks']

,loudness,tempo,tempo_confidence,time_signature,time_signature_confidence,key,key_confidence,mode,mode_confidence,danceability,...,tempo,type,id,track_href,duration_ms,time_signature,featuring_artists,track_name,track_popularity,available_markets
0,-8.077,77.468,0.081,4,0.997,9,0.061,0,0.226,0.626,...,77.468,audio_features,6IUiqtI8tE49sqGbmtrNd8,https://api.spotify.com/v1/tracks/6IUiqtI8tE49...,281293,4,NaN,There Goes My Baby,60,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B..."
0,-4.624,170.697,0.192,5,0.985,1,0.299,1,0.328,0.343,...,170.697,audio_features,5rXDTihGQ3TkYE3pmraAmH,https://api.spotify.com/v1/tracks/5rXDTihGQ3Tk...,238893,5,['Slash'],ROCKSTAR 101,51,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B..."
0,-7.268,127.901,0.617,4,1,5,0.277,0,0.382,0.684,...,127.901,audio_features,6TlRNJaezOdzdECnQeRuMM,https://api.spotify.com/v1/tracks/6TlRNJaezOdz...,275107,4,NaN,Bed,67,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B..."
0,-9.512,82.511,0.904,4,1,7,0.856,1,0.599,0.77,...,82.511,audio_features,1ckU1EhAO0Nr73QYw24SWJ,https://api.spotify.com/v1/tracks/1ckU1EhAO0Nr...,261573,4,NaN,My Love Is Your Love,70,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B..."
0,-4.253,115.044,0.076,5,0.995,1,0.474,1,0.374,0.619,...,115.044,audio_features,0BTq9LFqUdW7rGTq0LCKg1,https://api.spotify.com/v1/tracks/0BTq9LFqUdW7...,203468,5,NaN,Study People,13,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,-12.417,144.941,0.058,4,0.341,7,0.352,0,0.215,0.267,...,144.941,audio_features,1Zk8MAKaOSaAjDFbGVq02w,https://api.spotify.com/v1/tracks/1Zk8MAKaOSaA...,216827,4,NaN,Warhead - 12'' Version,14,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B..."
0,-11.235,120.266,0.171,4,1,5,0.748,1,0.697,0.419,...,120.266,audio_features,6W5B2atlbrHdC8vy8c3df8,https://api.spotify.com/v1/tracks/6W5B2atlbrHd...,248000,4,['Wings'],My Love,32,"[CA, US]"
0,-7.997,92.262,0.325,4,0.577,8,0.392,1,0.441,0.199,...,92.262,audio_features,5xFqDS7kANSCyzne6yqjIV,https://api.spotify.com/v1/tracks/5xFqDS7kANSC...,217333,4,NaN,Fight Till Death,43,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B..."
0,-4.937,144.532,0.011,4,0.984,11,0.331,0,0.364,0.252,...,144.532,audio_features,6OGkZoqso1wbIy5O6cV56N,https://api.spotify.com/v1/tracks/6OGkZoqso1wb...,530400,4,NaN,Heir Apparent,40,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B..."


In [37]:
pickle.dump(tables, open('tables.pkl', 'wb'))